In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
train_dataset = torchvision.datasets.MNIST(root = '.',
                                          download=True,
                                          train=True,
                                          transform=torchvision.transforms.ToTensor())

In [ ]:
dataloader = DataLoader(train_dataset,
                       batch_size=16,
                       shuffle=True,
                       num_workers=0,
                       drop_last=False,
                       pin_memory=True)

In [ ]:
class LogisticReg(torch.nn.Module):
    def __init__(self):
        super(LogisticReg, self).__init__()
        self.layer1 = torch.nn.Linear(28*28, 1500)
        ##ADD HIDDEN LAYERS HERE#######
        self.act = torch.nn.Sigmoid()
        
    def forward(self, x):
        ##MODIFY THIS FUNCTION ACCORDING TO THE HIDDEN LAYER DEFINITIONS###
        return self.act(self.layer1(x))

In [ ]:
Logistic_reg = LogisticReg()

In [ ]:
optimizer = torch.optim.Adam(Logistic_reg.parameters(), lr=0.008, betas=(0.9, 0.99))

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(100):
    for idx, (x1, y1) in enumerate(dataloader):
        x1 = x1.to('cuda')
        y1 = y1.to('cuda')
        optimizer.zero_grad()
        pred = Logistic_reg(x1.view(16, -1))
        lossval = loss_function(pred, y1)
        lossval.backward(retain_graph=True)
        optimizer.step()
        
    if epoch % 10 == 0:
        print("Loss: {}".format(lossval.item()))

In [ ]:
###To load the pretrained model
# log_reg = LogisticReg()
# log_reg.load_state_dict(torch.load('logistic_reg_2.pth'), strict=True)

In [ ]:
test_dataset = torchvision.datasets.MNIST(root = '.',
                                          download=True,
                                          train=False,
                                          transform=torchvision.transforms.ToTensor())

dataloader_test = DataLoader(test_dataset,
                       batch_size=1,
                       shuffle=False,
                       num_workers=0,
                       drop_last=False,
                       pin_memory=True)

In [ ]:
log_reg = log_reg.cpu()
log_reg.eval()
y_true = []
y_pred = []
for idx, (x1, y1) in enumerate(dataloader_test):
    y_true.append(y1.numpy())
    pred = log_reg(x1.view(1,-1))
    _, pred_val = torch.max(pred, 1)
    y_pred.append(pred_val.detach().numpy())

In [ ]:
#To save the trained model as .pth file
state_dict = Logistic_reg.state_dict()
for key, param in state_dict.items():
    state_dict[key] = param.cpu()
torch.save(state_dict, 'logistic_reg.pth')

In [ ]:
torchvision.transforms.ToPILImage()(x1[0]).show()

In [ ]:
pred

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
con_mat = confusion_matrix(y_true,y_pred)
plt.figure(figsize = (10,7))
sn.heatmap(con_mat, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')